# Direct Correction for IPDA / DIAL for ABLE

I want to evaluate how much a radiometric flux error translates to an error in a water vapor retrieval for something like IPDA:

Number concentration of water vapor (WV) particles $n_{WV}$ is directly related to the optical depth $\tau$. This comes out of
$$n_{WV}=\frac{1}{2\Delta\sigma_{abs}\Delta R}\ln\left(\frac{P_{OFF}-P_B}{P_{ON}-P_B}\right)=\frac{1}{\Delta\sigma_{abs}\Delta R}\Delta\tau$$
where $\sigma_{abs}$ is the absorption cross section, $P$ is the optical power, $\Delta R$ is the range resolution, and $\tau$ is the optical depth. Note that this expression does ignore other terms in the DIAL equation such as backscatter and extinction. Errors in the flux can be injected via
$$\Delta\tau=\frac{1}{2}\ln\left(\frac{P_{OFF}\pm\epsilon_{OFF}-P_B}{P_{ON}\pm\epsilon_{ON}-P_B}\right)$$
where the $\epsilon$ terms are the errors associated with the fluxes. For more advanced retrievals that also include terms like temperature (e.g., mixing ratio) that have there own associated error sources, it's important that the accuracy for $\tau$ is less than 5%. For most DIAL applications, use $\tau$ values between 1 - 2.
I will use fluxes $\Phi$ instead of powers (since the ratio maintains linearity $P=\Phi E$, where $E$ is energy per photon). Here, I assume the energy per photon for on and offline to be approximately equivalent... For ABLE, they expect an offline flux $\Phi_{OFF}=3$ MHz for dark albedo. This means $\Phi_{ON}=\frac{3e6}{e^2}=406$ kHZ.

Now for DIAL:
$$n_{WV}=\frac{1}{2\Delta\sigma_{abs}}\frac{d}{dR}\left(\ln\left[\frac{P_{OFF}-P_B}{P_{ON}-P_B}\right]\right)=\frac{1}{\Delta\sigma_{abs}}\frac{d}{dR}\Delta\tau$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import time
import xarray as xr
import pickle
import torch
from pathlib import Path

# import library for simulating Poisson point processes
cwd = os.getcwd()
dirLib = cwd + r'/library'
if dirLib not in sys.path:
    sys.path.append(dirLib)

# import sim_deadtime_utils as sim
from load_ARSENL_data import set_binwidth

import fit_polynomial_utils_cloud as fit
import data_organize as dorg

eps = sys.float_info.epsilon

C:\Users\Grant\anaconda3\envs\arsenl-conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tau_aim = 1  # Aiming for optical depth value of 1 
phi_off = 10e6  # [Hz] offline flux
phi_on = phi_off / np.e**(2*tau_aim)
print('Ideal Optical Depth: {}'.format(tau_aim))
print('Offline Flux: {:.3f} MHz'.format(phi_off/1e6))
print('Online Flux: {:.3f} MHz'.format(phi_on/1e6))

Ideal Optical Depth: 1
Offline Flux: 10.000 MHz
Online Flux: 1.353 MHz


In [3]:
eps_off = 871625  # [Hz] estimate for offline error in flux
eps_on = 45057  # [Hz] assume online is linear due to low flux

tau = 1/2*np.log((phi_off + eps_off)/(phi_on + eps_on))

print('Output Optical Depth: {:.3f}'.format(tau))
print('Error: {:.2f}%'.format(np.abs(tau-tau_aim)/tau_aim*100))

Output Optical Depth: 1.025
Error: 2.54%
